First we need to load the sequence file

In [1]:
from datasets import load_dataset

trainFile="Data/s288c.fasta.train.txt"
testFile="Data/s288c.fasta.test.txt"
    

Then we will need to convert each sequence to codons, each codon is treated as a "word" and the sequence is the "sentence"

In [2]:
dataset=load_dataset('text', data_files=trainFile,split='train')



Using custom data configuration default-f555aa9781812fb7
Reusing dataset text (/home/lu/.cache/huggingface/datasets/text/default-f555aa9781812fb7/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)


Now let's try to train a tokenlizer on our data

In [3]:
dataset

Dataset({
    features: ['text'],
    num_rows: 4990
})

In [4]:
batch_size = 1000
all_texts = [dataset[i : i + batch_size]["text"] for i in range(0, len(dataset), batch_size)]

In [5]:
batch_size = 1000
all_texts = [dataset[i : i + batch_size]["text"] for i in range(0, len(dataset), batch_size)]

def batch_iterator():
    for i in range(0, len(dataset), batch_size):
        yield dataset[i : i + batch_size]["text"]

In [6]:
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer

tokenizer = Tokenizer(models.WordPiece(unl_token="[UNK]"))

In [7]:
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=False)


In [8]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(),normalizers.StripAccents()]
)

In [9]:
tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()


In [10]:
tokenizer.pre_tokenizer.pre_tokenize_str("CAA GTG ACC AGA ATG ATC ACC GGT GTT")


[('CAA', (0, 3)),
 ('GTG', (4, 7)),
 ('ACC', (8, 11)),
 ('AGA', (12, 15)),
 ('ATG', (16, 19)),
 ('ATC', (20, 23)),
 ('ACC', (24, 27)),
 ('GGT', (28, 31)),
 ('GTT', (32, 35))]

In [11]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=5000, special_tokens=special_tokens)

In [12]:
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)


In [13]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

2 3


In [14]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", cls_token_id),
        ("[SEP]", sep_token_id),
    ],
)

In [15]:
encoding = tokenizer.encode("CAA GTG ACC AGA ATG ATC ACC GGT GTT")


In [ ]:
encoding.tokens

In [17]:
encoding.type_ids


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
tokenizer.decoder = decoders.WordPiece(prefix="##")
tokenizer.save("./yeast_token")

In [ ]:
from transformers import BertTokenizer

model_checkpoint="gpt2"
tokenizer = BertTokenizer.from_pretrained('./yeast_token')

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [25]:
datasets = load_dataset("text", data_files={"train": trainFile, "validation": testFile})
print(datasets)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])
print(tokenized_datasets)

Using custom data configuration default-858e7f4f9c294c37
Reusing dataset text (/home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 4990
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 1000
    })
})
 

Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-1bef48533eae86eb.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-a62fd433db4e3597.arrow


...

Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-9163a6e7b9339e6c.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-4b09c61881da5fc5.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-b7d2513df9da48ab.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-573865020246181d.arrow


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4990
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1000
    })
})


In [26]:
import pandas as pd
from datasets import Dataset

def tokenize_function(examples):
    return tokenizer(examples["text"])

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

# block_size = tokenizer.model_max_length
block_size = 128

tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

print(tokenized_datasets)

Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-1bef48533eae86eb.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-a62fd433db4e3597.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6be36851dd4a9875.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-c3e7ed778caa4ff0.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-9163a6e7b9339e6c.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-4b09c61881da5fc5.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-b7d2513df9da48ab.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-573865020246181d.arrow


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4990
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1000
    })
})


In [27]:
lm_datasets =tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)



Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-c4236cdd3c40af67.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-0f93d0da38e3bc78.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-2f8844b0474ae1c6.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-47afe6832616aaab.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-ddedf6a58972e36e.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-b6f8bbdb347be4a9.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-0283cc1d8fec3263.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-5d5e269f2bd535d4.arrow


In [31]:
lm_datasets["train"][1]



{'input_ids': [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  N

In [ ]:
print(lm_datasets["train"])
print(lm_datasets["train"][0])


In [ ]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    f"{model_checkpoint}-CUBERT",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [ ]:
trainer.train()